# User Groups

In [4]:
# import required packages
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [5]:
# config
ds_name = 'Bookcrossing'
ds_acronym = 'BX'
ds_core = '5Rate'
ds_feedback = 'Explicit'

In [6]:
# loading the Book-Crossing dataset
dataset = pd.read_csv(f"dataset/{ds_name}/{ds_acronym}-Book-{ds_feedback}-{ds_core}-Map.csv", sep=",")

In [7]:
# get user distribution
user_dist = dataset['uid'].value_counts()
num_users = len(user_dist)
print('No. users: ' + str(num_users))
print('Mean books per user: ' + str(user_dist.mean()))
print('Min books per user: ' + str(user_dist.min()))
print('Max books per user: ' + str(user_dist.max()))

No. users: 6358
Mean books per user: 13.927650204466813
Min books per user: 5
Max books per user: 153


In [8]:
# get item distribution
item_dist = dataset['bid'].value_counts()
num_items = len(item_dist)
print('No. items: ' + str(num_items))

No. items: 6921


In [9]:
# get top items
top_fraction = 0.2
num_top = int(top_fraction * num_items)
top_item_dist = item_dist[:num_top]
print('No. top items: ' + str(num_top))

No. top items: 1384


In [10]:
top_item_set = set(top_item_dist.keys())

In [11]:
user_books = defaultdict(set)
for eachline in tqdm(dataset.iterrows()):
    uid, bid, rating = eachline[1][0], eachline[1][1], eachline[1][2] 
    uid, bid = int(uid), int(bid)
    if uid in user_books.keys():
        user_books[uid].add(bid)
    else:
        user_books[uid] = {bid}

88552it [00:11, 7711.97it/s]


In [12]:
user_pop_book_ratio = {}

for user, books in tqdm(user_books.items()):
    if user not in user_pop_book_ratio.keys():
        user_pop_book_ratio[user] = (len(set(books) & set(top_item_set))) / len(set(books))

100%|██████████| 6358/6358 [00:00<00:00, 25075.92it/s]


In [13]:
sorted_user_pop_book_ratio = {k: v for k, v in sorted(user_pop_book_ratio.items(), key=lambda item: item[1], reverse=True)}

In [14]:
num_user = len(sorted_user_pop_book_ratio)
num_top_users = int(top_fraction * num_users)
num_top_users

1271

In [15]:
# Files to save users ids based on the number of checkins
inactive_users = open(f'dataset/{ds_name}/user-groups/inactive_users.txt', 'w')
inactive_users.write('user_id' + '\n')

medium_users = open(f'dataset/{ds_name}/user-groups/medium_users.txt', 'w')
medium_users.write('user_id' + '\n')

active_users = open(f'dataset/{ds_name}/user-groups/active_users.txt', 'w')
active_users.write('user_id' + '\n')

8

In [16]:
for uid in list(sorted_user_pop_book_ratio.keys())[:num_top_users]:
    active_users.write(str(uid) + '\n')
active_users.close()

In [17]:
for uid in list(sorted_user_pop_book_ratio.keys())[num_top_users:len(sorted_user_pop_book_ratio) - num_top_users]:
    medium_users.write(str(uid) + '\n')
medium_users.close()

In [18]:
for uid in list(sorted_user_pop_book_ratio.keys())[len(sorted_user_pop_book_ratio) - num_top_users:len(sorted_user_pop_book_ratio)]:
    inactive_users.write(str(uid) + '\n')
inactive_users.close()

In [19]:
# file closing
inactive_users.close()
medium_users.close()
active_users.close()